 ## Step 4: Tối ưu hóa MLP bằng GridSearchCV

 Chạy riêng bước này để tìm mô hình MLP tốt nhất cho tập dữ liệu Iris.

 ### 4.1: Chuẩn bị dữ liệu (tự động tạo lại iris_scaled.csv nếu chưa có)

In [7]:
import os
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler

if not os.path.exists("iris_scaled.csv"):
    iris = load_iris()
    data = pd.DataFrame(data=iris.data, columns=iris.feature_names)
    data['target'] = iris.target

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(data.drop('target', axis=1))
    data_scaled = pd.DataFrame(X_scaled, columns=iris.feature_names)
    data_scaled['target'] = data['target']

    data_scaled.to_csv('iris_scaled.csv', index=False)
    print("✅ Đã tạo file iris_scaled.csv từ dữ liệu gốc.")
else:
    print("✅ Đã tìm thấy file iris_scaled.csv.")


✅ Đã tìm thấy file iris_scaled.csv.


 ### 4.2: Cài đặt GridSearchCV với pipeline

In [8]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import joblib

# Đọc dữ liệu
df = pd.read_csv("iris_scaled.csv")
X = df.drop("target", axis=1)
y = df["target"]

# Tạo pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # (dự phòng nếu chạy raw dữ liệu)
    ('mlp', MLPClassifier(max_iter=1000, random_state=42))
])

# Lưới siêu tham số
param_grid = {
    'mlp__hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'mlp__activation': ['relu', 'tanh', 'logistic'],
    'mlp__solver': ['adam', 'sgd'],
    'mlp__alpha': [0.0001, 0.001, 0.01],
    'mlp__learning_rate_init': [0.001, 0.01],
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


 ### 4.3: Tiến hành GridSearchCV

In [9]:
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=cv,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2,
    return_train_score=True
)

print("⏳ Đang huấn luyện, vui lòng chờ...")
grid_search.fit(X, y)
print("✅ Huấn luyện hoàn tất.")


⏳ Đang huấn luyện, vui lòng chờ...
Fitting 5 folds for each of 144 candidates, totalling 720 fits
✅ Huấn luyện hoàn tất.


 ### 4.4: Lưu và hiển thị kết quả

In [10]:
# Lưu mô hình và kết quả
joblib.dump(grid_search, "mlp_gridsearch_model.pkl")
results_df = pd.DataFrame(grid_search.cv_results_)
results_df.to_csv("mlp_gridsearch_results.csv", index=False)

# In kết quả tốt nhất
print("🎯 Best Parameters:", grid_search.best_params_)
print(f"🏆 Best Accuracy: {grid_search.best_score_ * 100:.2f}%")


🎯 Best Parameters: {'mlp__activation': 'tanh', 'mlp__alpha': 0.0001, 'mlp__hidden_layer_sizes': (100, 50), 'mlp__learning_rate_init': 0.001, 'mlp__solver': 'adam'}
🏆 Best Accuracy: 97.33%
